<a href="https://colab.research.google.com/github/arjunparmar/TUNEX/blob/main/Deepanshu/Training/train_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import sys

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
data_Dir = "/content/drive/MyDrive/main"

In [6]:
import tensorflow as tf
from keras.preprocessing import image_dataset_from_directory
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

In [7]:
height = 224
width = 224
batch_Size = 32

In [8]:
train_Data = image_dataset_from_directory(
    data_Dir,
    labels="inferred",
    label_mode="categorical",
    batch_size=32,
    image_size=(height, width),
    shuffle=True,
    seed = 20,
    validation_split = 0.2,
    subset = "training",
)

Found 17676 files belonging to 7 classes.
Using 14141 files for training.


In [9]:
valid_Data = image_dataset_from_directory(
    data_Dir,
    labels="inferred",
    label_mode="categorical",
    batch_size=32,
    image_size=(height, width),
    shuffle=True,
    seed = 20,
    validation_split = 0.2,
    subset = "validation",
)

Found 17676 files belonging to 7 classes.
Using 3535 files for validation.


In [10]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_Tuned = train_Data.cache().prefetch(buffer_size=AUTOTUNE)
valid_Tuned = valid_Data.cache().prefetch(buffer_size=AUTOTUNE)

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers.experimental.preprocessing import Rescaling
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.regularizers import l2

In [13]:
model = Sequential()

model.add(Rescaling(1./255))
model.add(Conv2D(32, (3, 3), activation='relu', padding = 'same', input_shape=(height, width, 1)))
model.add(Conv2D(32, (3, 3), activation='relu', padding = 'same', input_shape=(height, width, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3), activation='relu', padding = 'same'))
model.add(Conv2D(64, (3, 3), activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.3))

#model.add(Conv2D(128, (3, 3), activation='relu', padding = 'same'))
#model.add(Conv2D(128, (3, 3), activation='relu'))
#model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
#model.add(Dropout(0.3))

#model.add(Conv2D(256, (3, 3), activation='relu', padding = 'same'))
#model.add(Conv2D(256, (3, 3), activation='relu'))
#model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
#model.add(Dropout(0.3))


model.add(Flatten())
model.add(Dense(256, activation='relu'))
#model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

#model.add(Dense(16, activation='relu'))
#model.add(BatchNormalization())
#model.add(Dropout(0.2))

model.add(Dense(7, activation='softmax'))

In [14]:
model.compile(
    optimizer = Adam(lr = 0.001),
    loss = "categorical_crossentropy",
    metrics = ['accuracy'],
)

In [15]:
early_Stop = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=3,
    verbose=1,
    restore_best_weights=True,
)

In [16]:
checkpoint = ModelCheckpoint(
    '/gdrive/MyDrive/model_One.h5',
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True,
)

In [17]:
callbacks = [early_Stop]

In [ ]:
model.fit(
    train_Tuned,
    epochs = 30,
    validation_data = valid_Tuned,
    callbacks = callbacks,
)

Epoch 1/30
442/442 [==============================] - 3791s 8s/step - loss: 2.4706 - accuracy: 0.1490 - val_loss: 1.9436 - val_accuracy: 0.1389
Epoch 2/30
442/442 [==============================] - 563s 1s/step - loss: 2.0453 - accuracy: 0.1548 - val_loss: 1.9225 - val_accuracy: 0.1901
Epoch 3/30
442/442 [==============================] - 565s 1s/step - loss: 1.9795 - accuracy: 0.1730 - val_loss: 1.8935 - val_accuracy: 0.2164
Epoch 4/30
442/442 [==============================] - 559s 1s/step - loss: 1.9097 - accuracy: 0.2074 - val_loss: 1.8940 - val_accuracy: 0.2062
Epoch 5/30
442/442 [==============================] - 564s 1s/step - loss: 1.7853 - accuracy: 0.2806 - val_loss: 1.4403 - val_accuracy: 0.5010
Epoch 6/30
442/442 [==============================] - 565s 1s/step - loss: 1.3947 - accuracy: 0.4712 - val_loss: 0.9998 - val_accuracy: 0.6504
Epoch 7/30
442/442 [==============================] - 568s 1s/step - loss: 1.0455 - accuracy: 0.6045 - val_loss: 0.7744 - val_accuracy: 0.721